In [1]:
import os
import sys
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib import ticker
import datetime as dt
import parameters as p


In [5]:
def main():

    # input parameters
    
    # leaf area index(m2 m-2) 
    LAI       =  
    
    # height above the soil surface of U_x
    z         =  

    # the saturated specific humidity at the surface temperature
    qsat      = function(Tsrf)    
    
    # (kg kg-1) specific humidity of the canopy air ;  the specific humidity of the atmosphere    
    qa        =  
    
    #(m s-1) observed wind speed away from the surface and canopy 
    Uair      = 

    # friction velocity
    Ux        = 
    '''
    Ux = make it as a input
            psim_1 = psim(canopy%zetar(:,iter)*rough%zref_uv/rough%zref_tq)
            rescale = C%VONK * MAX(met%ua,C%UMIN)
            z_eff = rough%zref_uv / rough%z0m
            psim_arg = canopy%zetar(:,iter) * rough%z0m / rough%zref_tq
            psim_2 = psim( psim_arg )
            lower_limit = rescale / ( LOG(z_eff) - psim_1 + psim_2 )
            canopy%us = MIN(MAX(1.e-6, lower_limit ), 10.0 )
    '''
    
    theta     = # wb



    # (m s-1) canopy friction velocity 
    U_x = Ux*exp(Cv*(1.-z/h)) 
    # U_x = canopy%us(i)*exp(-rough%coexp(i)*(1.0-canopy%sublayer_dz(i)/max(1e-2,rough%hruff(i))))

    # rough%disp    (m) the displacement height of the canopy  
    d  = (1.0-(1.0-np.exp(-1.0*np.sqrt(15.*max((L*0.5),0.0005))))/(np.sqrt(15.*max((L*0.5),0.0005))))*p.h          
    # xx = SQRT( 15.0 * MAX( ( canopy%rghlai * 0.5 ), 0.0005 ) )
    # dh = 1.0-(1.0 - EXP(-xx ))/xx, eq.8 Raupach 1994, BLM, vol 71, p211-216:
    # rough%disp = dh * rough%hruff, calculate zero-plane displacement:

    # rough%z0soilsn (m) soil surface roughness height, turbulent airflow's length scale
    Z0s = 0.01*L + 0.02*U_x**2/p.g 
    #rough%z0soil   = 0.01*min(1.0,canopy%vlaiw) + 0.02*min(canopy%us**2/C%GRAV,1.0)
    #rough%z0soilsn = max(1.e-7,rough%z0soil) 
    
    
    rg = calc_rg(LAI, U_x, d, Z0s)
    rsv = calc_rsv(theta, Tsrf, qsat)
    rBL = calc_rBL(Uair, U_x, theta_srf, tvair)
    Esoil = calc_esoil(pres, tvair, qsrf, qa, rg, rsv, rBL)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 58)

In [ ]:
def calc_rg(LAI, U_x, d, Z0s):
    '''
    calculate Rg  
    '''
    
    # canopy%rghlai (m) LAI over the whole grid cell, it is not Monin-Obhukov length,Decker 2017 made a mistake  
    L  = LAI 
    # canopy%rghlai = veg%vlai * rough%hruff / MAX( 0.01, veg%hc )

    # sparseness factor (see p.7 in CABLE guide 2016), =1 for bare ground; > 1 when LAI > 1
    fsp = 1./ max(2/3*p.h/d, 1.) 
    # The larger LAI -> the larger d (d always smaller than h) -> the larger fsp, but fsp cannot larger than 1
    # that is to say, when LAI lower than 1 -> fsp lower than 1 -> rg will increase -> harder to evaporate
    
    # the resistance for both the sensible and latent heat fluxes from the soil to the canopy air 
    rg  = np.log(d/Z0s)*(np.exp(2.*p.Csw*L)-np.exp((2.*p.Csw*L) *(1 -d/p.h)))/(2.*p.a_3**2*fsp*p.Ctl*p.Csw*L)
    # rough%rt0us  = (MAX((2./3.)*rough%hruff/rough%disp,1.0))*LOG(rough%disp/rough%z0soilsn)*
    #                (EXP(2*C%CSW*canopy%rghlai)-EXP(2*C%CSW*canopy%rghlai*(1-rough%disp/rough%hruff)))/
    #                (C%A33**2 * C%CTL * 2 * C%CSW * canopy%rghlai)
    # ssnow%rtsoil(j) = max(rt_min,rough%rt0us) / canopy%us
    
    return rg

In [ ]:
def calc_rsv(theta, Tsrf, qsat):
    '''
    calculate rsv  
    '''
    # calculate rsv
    qsat
    Tsrf      = function(K) # soil surface temperature

    
    # relative wetness at the surface
    theta_srf = (theta - p.theta_r)/(p.theta_sat - p.theta_r) 
    
    # (kg kg-1) specific humidity at the soil surface 
    qsrf = qsat*np.exp((p.Phi_1*p.g)/(p.Rwv*Tsrf))  

    # (m s-1) hydraulic conductivity of the soil at the surface
    K = 0.001*p.hyds*theta_srf**(2.*p.bch+3.) 
    # hk_zero(i) = max(0.001*soil%hyds_vec(i,1)*(min(max(rel_s(i),0.001_r_2),1._r_2)**(2._r_2*soil%bch_vec(i,1)+3._r_2) ),1e-12)

    rsv   = p.gamma/(4.*K)
    # rsv = lm/ (4.0*hk_zero(i))
    
    return rsv

In [ ]:
def calc_rBL(Uair, U_x, theta_srf, tvair):
    '''
    calculate rBL
    '''
    
    # time spectrum parameter
    alpha = 0.3/U_x*(Uair - 1) 
    #eddy_shape(i) = 0.3*met%ua(i)/ max(1.0e-4, max(1.0e-3,canopy%us(i))* exp(-rough%coexp(i)*(1.0-canopy%sublayer_dz(i) / max(1e-2,rough%hruff(i))) ) )
    
    Chi   = 1
    for n in np.arange(1,alpha,1):
        Chi  = Chi*(2.*alpha+1.)
    Chi = Chi * (2.*(alpha -n)+1)

    Gamma = my_gamma(alpha +1)
    
    # eddy_mod, dimensionless coefficient dependent on eddy residence time spectrum parameter a (-)
    G    = 2.2 * np.sqrt(112.* p.Pi)/Gamma/ (2**(alpha + 1)*np.sqrt(alpha +1))* Chi
    #eddy_mod(i) = 2.2*sqrt(112.0*(pi_r_2)) / (2.0**(eddy_shape(i)+1.0) * sqrt(eddy_shape(i)+1.0))

??? # extinction coefficient for wind speed in the canopy
    Cv        = - rough%coexp ??? (= - rough%usuh / ( C%VONK * C%CCW_C * ( 1.0 - dh ) ))


    #(m2 s-1) the kinematic viscosity of air
    v = 0.00001 * (1.35 + 0.0092 * (tvair - 273.16)) 
    # air%visc = 1e-5 * MAX(1.0, 1.35 + 0.0092 * (met%tvair - C%TFRZ) )
    # tfrz     = 273.16 ! Temp (K) corresp. to 0 C
    
    # depth of the diffusion dominated region at the soil-air interface, within delta molecular diffusion dominates transport 
    delta  = G*v/U_x 
    #canopy%sublayer_dz(i) = min(0.05, max(eddy_mod(i) * air%visc(i) / max(1.0e-3,canopy%us(i))*
    #                        exp(-rough%coexp(i)*(1.0-canopy%sublayer_dz(i)/max(1e-2,rough%hruff(i)))),1e-7)  )

    R = 2.*p.sigma/(p.rho*p.g*p.Phi_sat) # pore radius, using Young-Laplace equation
    # pore_radius = 0.148/(1000.0*9.81*abs(soil%sucs_vec(:,1))/1000.0) 

    F = 1./(p.Pi*np.sqrt(theta_srf))*(np.sqrt(p.Pi/(4.*theta_srf))-1) 
            # soil_moisture_mod, surface wetness dependency model from Schlunder (1988), 
            # the asymptotic solution to constant rate diffusion from a partly wetted surface 
    # soil_moisture_mod(i) = 1.0/(pi_r_2)/sqrt(wb_liq(i))* ( sqrt((pi_r_2)/(4.0*wb_liq(i)))-1.0)

    rBL   = (delta + R*np.sqrt(p.Pi)*F)/p.D
    # rBL = (canopy%sublayer_dz(i)+ pore_size(i) * soil_moisture_mod(i)) / rt_Dff


    def my_gamma(a, c_gamma) 
        gamma_pre = [0.99999999999980993, 676.5203681218851, -1259.1392167224028,\
                     771.32342877765313, -176.61502916214059, 12.507343278686905,\
                     -0.13857109526572012, 9.9843695780195716e-6, 1.5056327351493116e-7]
        c_gamma = 7
        x = a
        if ( x < 0.5 ):
            g = (Pi) / ( np.sin((Pi)*x) * my_gamma(1.0-x) )
        else:
            x = x - 1.0
            t = gamma_pre[0]
            for i in np.arange(1, c_gamma+2 +1 ):
                t = t + gamma_pre(i-1)/(x+i)
            w = x + c_gamma + 0.5
            g = np.sqrt(2.0*Pi * w**(x+0.5) * np.exp(-w) * t
        return g


In [1]:
def calc_esoil(pres, tvair, qsrf, qa, rg, rsv, rBL):
    '''
    calculate Esoil
    ''' 

    pres =  # met%pmb * 100.0 

    qsrf - qa =      # dq_unsat = ssnow%rh_srf*ssnow%qstss - met%qvair
    
    # (kg m-3)density of dry air 
    rhoa =  min(1.3, 0.02897*pres/(8.3143* tvair))  
    # air%rho = MIN(1.3, C%RMAIR * (met%pmb * 100.0 / (C%RGAS * (met%tvair))))
    # rmair  = 0.02897, molecular wt: dry air (kg/mol)
    # rgas   = 8.3143,  universal gas const  (J/mol/K)

    Esoil = rhoa * p.Lv * (qsrf - qa)/(rg + Z0s/delta*(rsv+rBL))
    # ssnowpotev =air%rho * air%rlam * dq /ssnow%rtsoil
     rough%z0soil(i)/canopy%sublayer_dz(i)



SyntaxError: invalid syntax (<ipython-input-1-53edc83215ea>, line 9)